In [1]:
# import libraries

import os
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import mygene
import sklearn.preprocessing as sk
import seaborn as sns
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from torch.utils.data.sampler import WeightedRandomSampler
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score, precision_recall_curve
import random
from random import randint
from sklearn.model_selection import StratifiedKFold
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
# define directories

cell_line_dir = "/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC/"
models_dir = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/PDX_EGFRi/Expression_CNA/Models/"
DEGs_dir = "/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/EGFRi/"
dataset_dir = "/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/"
pdx_data_dir = "/Volumes/Expansion/Thesis Work/Datasets/PDX/Expression/"
save_results_to = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/PDX_EGFRi/Expression_CNA/Predictions/"
save_figures_to = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/PDX_EGFRi/Expression_CNA/Figures/"

In [3]:
# set random seeds

torch.manual_seed(42)
random.seed(42)

In [4]:
# change directory to read GDSC cell line details

os.chdir(cell_line_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC


In [5]:
# read GDSC cell line details table

GDSC_cell_line_details = pd.read_excel("GDSC_Cell_Lines_Details.xlsx", keep_default_na = False)
GDSC_cell_line_details.set_index("COSMIC identifier", inplace = True)
GDSC_cell_line_details.tail()

,Sample Name,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
COSMIC identifier,,,,,,,,,,,,
1298218,KP-2,Y,N,Y,N,Y,pancreas,pancreas,PAAD,,D/F12,Adherent
1330932,KO52,Y,Y,N,N,N,leukemia,acute_myeloid_leukaemia,LAML,MSS/MSI-L,D/F12,NA
1331030,SC-1,Y,Y,N,N,N,lymphoma,B_cell_lymphoma,DLBC,MSS/MSI-L,R,NA
1503373,U-CH2,Y,Y,N,N,N,bone,bone_other,,MSS/MSI-L,D/F12,Adherent
,TOTAL:,1001,996,968,957,990,,,,,,


In [6]:
# change directory to read GDSC expression dataset (EGFRi)

os.chdir(dataset_dir + "/exprs/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs


In [7]:
# read GDSC expression dataset (EGFRi) 

GDSCE = pd.read_csv("GDSC_exprs.z.EGFRi.tsv", 
                    sep = "\t", index_col=0, decimal = ",")
GDSCE = pd.DataFrame.transpose(GDSCE)
GDSCE.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,107984199,107984208,107984325,107984411,107984648,107984889,107984989,107986313,107986782,107986810
683665,-0.4711563767951986,-0.1857522957766964,0.9702409548662462,-0.4081281805960821,-0.4161511169999803,-0.4384159723467665,-1.154692246005183,0.5262132484107975,-0.7950924576033422,-0.6337442716864214,...,1.283534690590172,-0.3367265464586691,-0.0039762988723447,0.7378729872604209,0.9757614264575688,0.1494417647095801,2.062294063911948,-0.8140636761719959,0.2870333828833719,0.9524265085537148
683667,0.8924335646616343,-0.2118047645974752,-1.5249963829654445,-0.5512338119420259,-0.1375180480622428,-0.5276262991742051,-0.0305604198225781,0.5226216566753464,0.4877783097403741,0.2252883573642275,...,-0.0156845434045267,-0.1612071417519457,1.2033249282722671,2.7714836839186687,-0.3165072496327769,-0.8849200030571541,-0.5104734344593952,0.2984361940989172,-0.4349407065496763,0.4747633022523675
684057,0.6510004960254864,1.6578876382433665,-0.4977106390881842,-0.1212172092933644,-0.398455281697677,-0.2804901106672752,0.6455898116420014,0.215833091774836,-0.1723595788875009,-0.2170870770552223,...,-0.024863133395904,0.4073439017665206,-0.4440356638831038,0.8630347078663075,1.4486171428489227,-1.4777214830771732,0.5220400480464272,0.6441211958947753,0.9057518606131394,0.3018783753479829


In [8]:
# change directory to read GDSC CNA dataset (EGFRi)

os.chdir(dataset_dir + "/CNA/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/CNA


In [9]:
# read GDSC CNA dataset (EGFRi)

GDSCC = pd.read_csv("GDSC_CNA.EGFRi.tsv",
                    sep="\t", index_col=0, decimal=".")
GDSCC.drop_duplicates(keep='last')
GDSCC = pd.DataFrame.transpose(GDSCC)
GDSCC = GDSCC.loc[:, ~GDSCC.columns.duplicated()]

GDSCC.head()

gene_id,1,2,9,10,12,13,14,15,16,18,...,107985535,107985759,107986588,107986809,107986898,107987337,107987341,109731405,112441434,121676927
683665,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.00,1.807355,0.000000,0.000000,0.000000
683667,0.000000,0.0,0.321928,0.321928,-1.000000,-0.415037,0.0,0.0,-0.415037,-0.415037,...,-0.415037,-0.415037,0.000000,0.0,0.321928,0.00,-1.000000,-0.415037,0.000000,0.321928
684057,-0.415037,0.0,0.584963,0.584963,-1.000000,0.000000,0.0,0.0,-0.415037,-0.415037,...,-0.415037,0.000000,0.000000,0.0,0.807355,-1.00,-1.000000,-0.415037,-0.415037,0.807355
684059,0.000000,0.0,0.000000,0.000000,0.000000,0.415037,0.0,0.0,0.000000,0.415037,...,0.000000,0.000000,0.415037,0.0,0.415037,-4.32,-4.320000,0.000000,0.000000,0.736966
684062,-0.415037,0.0,0.321928,0.321928,-0.415037,-0.415037,0.0,0.0,-0.415037,0.000000,...,-0.415037,0.000000,0.000000,0.0,0.321928,-4.32,-4.320000,-0.415037,-1.000000,0.321928


In [10]:
# change directory to read GDSC response dataset (EGFRi)

os.chdir(dataset_dir + "/response/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response


In [11]:
# read GDSC response dataset (EGFRi)

GDSCR = pd.read_csv("GDSC_response.EGFRi.tsv",
                    sep="\t",
                    index_col=0,
                    decimal=",")
GDSCR.dropna(inplace=True)
GDSCR.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
GDSCR["response"] = GDSCR.loc[:, "response"].apply(lambda x: d[x])

GDSCR.head()

,response,logIC50,drug,exprs,CNA,mutations
sample_name,,,,,,
683665,0,1.49001536063508,Afatinib,1,1,1
683665,0,3.70042086392846,Afatinib (rescreen),1,1,1
683665,0,6.29444657935625,Cetuximab,1,1,1
683665,0,-1.50094791622811,CUDC-101,1,1,1
683665,1,-0.966773876200563,EKB-569,1,1,1


In [12]:
# print count of cell lines for each drug (EGFRi)

GDSCR["drug"].value_counts()

drug
EKB-569                770
Afatinib (rescreen)    757
CUDC-101               754
Afatinib               738
Gefitinib              734
Cetuximab              725
HG-5-88-01             422
Lapatinib              371
Erlotinib              362
Name: count, dtype: int64

In [13]:
# change directory to read PDX expression dataset homogenized with GDSC expression dataset (Erlotinib)

os.chdir(dataset_dir + "/exprs_homogenized/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs_homogenized


In [14]:
# read PDX expression dataset homogenized with GDSC expression dataset (Erlotinib)

PDXEerlotinib = pd.read_csv("PDX_exprs.Erlotinib.eb_with.GDSC_exprs.Erlotinib.tsv",
                   sep = "\t", index_col=0, decimal = ",")
PDXEerlotinib = pd.DataFrame.transpose(PDXEerlotinib)
PDXEerlotinib.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,100507206,100507254,100507436,100507472,100526773,100527978,100532746,100820829,102724473,105375355
X-1156,6.11861088229939,6.52864101100101,6.79951417489383,2.75014153880996,2.46025530503934,2.55715969202243,8.28052725761881,2.57788329077075,6.95887775544869,3.61452184934982,...,2.71395625069168,3.25738747236556,6.03809972147297,2.98821567577099,2.98369076792706,2.351997223295,2.52761397843368,5.09565670825996,4.1270944614262,2.42677851551937
X-1172,3.31280391584354,7.01865849503627,5.64214585248198,2.99828359748804,3.6752940409957,4.99060571336846,8.67761758763766,2.846798443615,9.99801881476899,3.81384234557669,...,2.76551893238154,3.3106737035931,4.94017287575563,2.98821567577099,2.98369076792706,2.351997223295,2.52761397843368,2.62204355997836,4.41769048689105,2.38094556877352
X-1323,5.21916353176773,3.70763705485075,6.05754416508987,2.75014153880996,3.00134646136568,2.46294977024261,7.39844941299792,3.10981562039544,10.013301293327,3.13578724141655,...,2.62947211484532,3.1747527931153,5.5262091243784,4.12076583575454,2.98369076792706,2.351997223295,2.52761397843368,2.62204355997836,4.02209193327285,2.27747617313197


In [15]:
# change directory to read PDX CNA dataset (Erlotinib)

os.chdir(dataset_dir + "CNA")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/CNA'

In [16]:
# read PDX CNA dataset (Erlotinib)

PDXCerlotinib = pd.read_csv("PDX_CNA.Erlotinib.tsv",
                        sep="\t", index_col=0, decimal=",")
PDXCerlotinib.drop_duplicates(keep='last')
PDXCerlotinib = pd.DataFrame.transpose(PDXCerlotinib)
PDXCerlotinib = PDXCerlotinib.loc[:, ~PDXCerlotinib.columns.duplicated()]

PDXCerlotinib.head(3)

ENTREZID,1,2,3,9,10,12,13,14,15,16,...,101340252,102723547,102724473,103091865,105375355,107126285,109623460,109731405,118126072,128385369
X-1156,0.9781956296816516,0.7004397181410922,0.7004397181410922,-0.2429767534925404,-0.2429767534925404,-0.2863041851566411,0.5607149544744789,0.3895668117627256,-0.2688167584278,-0.2688167584278,...,0.4594316186372972,0.0,-0.4246876693125631,-0.7369655941662062,0.0,1.100977647724821,-0.3770696490798233,-0.3219280948873623,-0.6551715030025588,0.0
X-1172,0.0,0.0,0.0,-0.5145731728297583,-0.5145731728297583,0.0,0.4276061727818994,0.4222330006830478,0.9297909977185974,0.0,...,-0.5994620704162712,-0.6780719051126377,0.3895668117627256,-0.4639470997597902,0.859969548221026,0.5109619192773793,-0.2863041851566411,0.0,0.0,1.0214797274104517
X-1323,-0.5670405927238937,0.3276873641760471,0.3276873641760471,-1.286304185156641,-1.286304185156641,0.0,0.0,0.4382928515791468,0.0,0.3103401206121505,...,0.0,-1.3040061868900998,-1.434402824145775,0.0,0.5993177936982261,0.0,-0.5994620704162712,0.3103401206121505,-0.5777669993169522,0.4382928515791468


In [17]:
# variance threshold for GDSC expression dataset (EGFRi)

selector = VarianceThreshold(0.05)
selector.fit_transform(GDSCE)
GDSCE = GDSCE[GDSCE.columns[selector.get_support(indices=True)]]

In [18]:
# fill NA values and binarize GDSC CNA dataset (EGFRi)

GDSCC = GDSCC.fillna(0)
GDSCC[GDSCC != 0.0] = 1

In [19]:
# select shared genes between GDSC and PDX expression and CNA datasets 

ls = GDSCE.columns.intersection(GDSCC.columns)
ls = ls.intersection(PDXEerlotinib.columns)
ls = ls.intersection(PDXCerlotinib.columns)

ls = pd.unique(ls)

print(f"GDSC and PDX # of common genes: {ls.shape[0]}")

GDSC and PDX # of common genes: 17901


In [20]:
# select shared samples between GDSC expression and response datasets (EGFRi)

ls2 = GDSCE.index.intersection(GDSCC.index)
ls2 = ls2.intersection(GDSCR.index)

print(f"GDSC # of common cell lines: {ls2.shape[0]}")

GDSC # of common cell lines: 800


In [21]:
# subset shared genes and samples in GDSC expression, CNA, and response datasets

GDSCE = GDSCE.loc[ls2, ls]
GDSCC = GDSCC.loc[ls2, ls]
GDSCR = GDSCR.loc[ls2, :]

print(f"GDSC # of common samples and genes (Expression): {GDSCE.shape}")
print(f"GDSC # of common samples and genes (CNA): {GDSCC.shape}")
print(f"GDSC # of samples (Response): {GDSCR.shape[0]}")

# There exists same sample names for different drugs, so row shape is different in GDSC response data

GDSC # of common samples and genes (Expression): (800, 17901)
GDSC # of common samples and genes (CNA): (800, 17901)
GDSC # of samples (Response): 5633


In [22]:
# select shared samples between PDX expression and CNA datasets (Erlotinib) 

ls3 = PDXEerlotinib.index.intersection(PDXCerlotinib.index)

print(f"PDX # of common samples (Erlotinib): {ls3.shape[0]}")

PDX # of common samples (Erlotinib): 21


In [23]:
# select shared genes and samples between PDX expression and CNA datasets (Erlotinib)

PDXEerlotinib = PDXEerlotinib.loc[ls3, ls]
PDXCerlotinib = PDXCerlotinib.loc[ls3, ls]

print(f"PDX # of common samples and genes for Expression (Erlotinib): {PDXEerlotinib.shape}")
print(f"PDX # of common samples and genes for CNA (Erlotinib): {PDXCerlotinib.shape}\n")

PDX # of common samples and genes for Expression (Erlotinib): (21, 17901)
PDX # of common samples and genes for CNA (Erlotinib): (21, 17901)



In [24]:
# change directory to read DEGs (EGFRi)

os.chdir(DEGs_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/EGFRi


In [25]:
# read DEGs (EGFRi)

DEGs_filtered_data = pd.read_excel("EGFRi_Differentially_Expressed_Genes (EnsemblID).xlsx",
                                    sheet_name = "Common DEGs")
print(f"There are {DEGs_filtered_data.shape[0]} DEGs (p-adjusted < 0.05 and |logFC| > 1) among EGFR signaling inhibitors\n")

DEGs_filtered_data.head()

There are 1028 DEGs (p-adjusted < 0.05 and |logFC| > 1) among EGFR signaling inhibitors



,Gene Symbol,Frequency
0,MYOF,9
1,TM4SF1,8
2,KRT18,8
3,WWC1,8
4,CAV1,8


In [26]:
# get gene symbol from mygene database

mg = mygene.MyGeneInfo()
DEGs_entrez_id = mg.querymany(DEGs_filtered_data["Gene Symbol"], 
                              species="human", 
                              scopes = "symbol",
                              field = "entrezgene",
                              as_dataframe=True)["entrezgene"]
DEGs_entrez_id = pd.unique(DEGs_entrez_id.dropna())
DEGs_entrez_id.shape[0]
DEGs_entrez_id = pd.Series(DEGs_entrez_id)

result = mg.query("SLC22A18", species="human", scopes="symbol", fields="entrezgene")
DEGs_entrez_id = pd.concat([DEGs_entrez_id, pd.Series(str(result["hits"][1]["entrezgene"]))], ignore_index=True)

DEGs_entrez_id.shape[0]

2 input query terms found dup hits:	[('CLECL1P', 2), ('CAST', 2)]
1 input query terms found no hit:	['SLC22A18']


1028

In [27]:
# assign GDSC datasets to new variables

exprs_z = GDSCE
cna = GDSCC
responses = GDSCR

In [28]:
# GDSC response dataset

responses.head(3)

,response,logIC50,drug,exprs,CNA,mutations
683665,0,1.49001536063508,Afatinib,1,1,1
683665,0,3.70042086392846,Afatinib (rescreen),1,1,1
683665,0,6.29444657935625,Cetuximab,1,1,1


In [29]:
# list drug names in GDSC response dataset

drugs = set(responses["drug"].values)
drugs

{'Afatinib',
 'Afatinib (rescreen)',
 'CUDC-101',
 'Cetuximab',
 'EKB-569',
 'Erlotinib',
 'Gefitinib',
 'HG-5-88-01',
 'Lapatinib'}

In [30]:
# filter GDSC expression and CNA datasets (EGFRi) as to drugs
# subset selected DEGs

expression_zscores = []
CNA = []
for drug in drugs:
    samples = responses.loc[responses["drug"] == drug, :].index.values
    e_z = exprs_z.loc[samples, :]
    c = cna.loc[samples, :]
    expression_zscores.append(e_z)
    CNA.append(c)

GDSCEv2 = pd.concat(expression_zscores, axis=0)
GDSCCv2 = pd.concat(CNA, axis=0)
GDSCRv2 = responses

ls7 = list(set(GDSCE.columns).intersection(set(DEGs_entrez_id.astype(int))))
ls8 = GDSCEv2.index.intersection(GDSCCv2.index)

GDSCEv2 = GDSCEv2.loc[ls8, ls7]
GDSCCv2 = GDSCCv2.loc[ls8, ls7]
GDSCRv2 = GDSCRv2.loc[ls8, :]

PDXEerlotinib = PDXEerlotinib.loc[:,ls7]
PDXCerlotinib = PDXCerlotinib.loc[:,ls7]

responses.index = responses.index.values + "_" + responses["drug"].values

print(f"GDSC # of common samples and genes (Expression): {GDSCEv2.shape}")
print(f"GDSC # of common samples and genes (CNA): {GDSCCv2.shape}")
print(f"GDSC # of common samples (Response): {GDSCRv2.shape[0]}\n")

print(f"PDX # of common samples and genes for Erlotinib (Expression): {PDXEerlotinib.shape}")
print(f"PDX # of common samples and genes for Erlotinib (CNA): {PDXCerlotinib.shape}\n")

GDSC # of common samples and genes (Expression): (5633, 1019)
GDSC # of common samples and genes (CNA): (5633, 1019)
GDSC # of common samples (Response): 5633

PDX # of common samples and genes for Erlotinib (Expression): (21, 1019)
PDX # of common samples and genes for Erlotinib (CNA): (21, 1019)



In [31]:
# GDSC response values

Y = GDSCRv2['response'].values
print(Y)

[0 0 0 ... 1 0 0]


In [32]:
# change directory to read PDX response dataset (Erlotinib)

os.chdir(dataset_dir + "response")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response'

In [33]:
# read PDX response dataset (Erlotinib)

PDXRerlotinib = pd.read_csv("PDX_response.Erlotinib.tsv", 
                      sep = "\t", 
                      index_col=0, 
                      decimal = ",")
PDXRerlotinib.dropna(inplace=True)
PDXRerlotinib.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
PDXRerlotinib["response"] = PDXRerlotinib.loc[:, "response"].apply(lambda x: d[x])
Ytserlotinib = PDXRerlotinib["response"].values    

print(f"There are {Ytserlotinib.shape[0]} samples with response data in the PDX (Erlotinib)\n")
Ytserlotinib

There are 21 samples with response data in the PDX (Erlotinib)



array([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [34]:
# read PDX dataset to select cohorts (Erlotinib)

pdx_cohort = pd.read_excel(pdx_data_dir + "/nm.3954-S2.xlsx",
                              "PCT raw data")
pdx_cohort = pdx_cohort.loc[pdx_cohort["Treatment"] == "erlotinib",]
pdx_cohort = pdx_cohort.loc[:,["Model","Tumor Type"]]
pdx_cohort = pdx_cohort.drop_duplicates()
pdx_cohort.set_index("Model", inplace = True) 
pdx_cohort.head()

,Tumor Type
Model,
X-1156,NSCLC
X-1172,NSCLC
X-1189,NSCLC
X-1228,NSCLC
X-1323,NSCLC


In [35]:
# merged PDX cohort table with PDX response dataset (Erlotinib)

dat_pdx = pd.merge(pdx_cohort,
                   PDXRerlotinib,
                   left_index = True,
                   right_index = True)
dat_pdx["Tumor Type"].value_counts()

Tumor Type
NSCLC    21
Name: count, dtype: int64

In [36]:
# filter NSCLC cohort samples (Erlotinib)

filter = (dat_pdx["Tumor Type"] == "NSCLC")
pdx_nsclc_samples = dat_pdx.loc[filter,].index
pdx_nsclc_samples

Index(['X-1156', 'X-1172', 'X-1323', 'X-1442', 'X-1499', 'X-1586', 'X-1658',
       'X-1683', 'X-1725', 'X-1787', 'X-1823', 'X-1934', 'X-1993', 'X-2017',
       'X-2042', 'X-2088', 'X-2094', 'X-3029', 'X-3237', 'X-3843', 'X-4819'],
      dtype='object')

In [37]:
# filter responses of NSCLC cohort samples (Erlotinib)

Ytserlotinib_nsclc = PDXRerlotinib.loc[pdx_nsclc_samples,"response"].values
Ytserlotinib_nsclc

array([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [38]:
# count of sensitive and resistance NSCLC samples (Erlotinib)

PDXRerlotinib.loc[pdx_nsclc_samples,"response"].value_counts()

response
0    18
1     3
Name: count, dtype: int64

In [39]:
# define maximum iteration

max_iter = 50

In [40]:
# assign number of neurons and dropout rates found for layers from the saved models

hdm1 = 512
hdm2 = 256
rate1 = 0.5
rate2 = 0.8
rate3 = 0.8

In [41]:
# load pre-trained modules and make predictions

scalerGDSC = sk.StandardScaler()
scalerGDSC.fit(GDSCEv2.values)
X_trainE = scalerGDSC.transform(GDSCEv2.values)
X_testEerlotinib = scalerGDSC.transform(PDXEerlotinib.values) 
X_testEerlotinib_nsclc = scalerGDSC.transform(PDXEerlotinib.loc[pdx_nsclc_samples,].values) 

X_trainC = np.nan_to_num(GDSCCv2.values)
X_testCerlotinib = np.nan_to_num(PDXCerlotinib.values.astype(np.float32))
X_testCerlotinib_nsclc = np.nan_to_num(PDXCerlotinib.loc[pdx_nsclc_samples,].values.astype(np.float32))

TX_testEerlotinib = torch.FloatTensor(X_testEerlotinib)
TX_testEerlotinib_nsclc = torch.FloatTensor(X_testEerlotinib_nsclc)

TX_testCerlotinib = torch.FloatTensor(X_testCerlotinib.astype(np.float32))
TX_testCerlotinib_nsclc = torch.FloatTensor(X_testCerlotinib_nsclc.astype(np.float32))

ty_testEerlotinib = torch.FloatTensor(Ytserlotinib.astype(int))
ty_testEerlotinib_nsclc = torch.FloatTensor(Ytserlotinib_nsclc.astype(int))


n_sampE, IE_dim = X_trainE.shape
n_sampC, IC_dim = X_trainC.shape

h_dim1 = hdm1
h_dim2 = hdm2
Z_in = h_dim1 + h_dim2 

class AEE(nn.Module):
    def __init__(self):
        super(AEE, self).__init__()
        self.EnE = torch.nn.Sequential(
            nn.Linear(IE_dim, h_dim1),
            nn.BatchNorm1d(h_dim1),
            nn.ReLU(),

            nn.Dropout(rate1))
    def forward(self, x):
        output = self.EnE(x)
        return output  


class AEC(nn.Module):
    def __init__(self):
        super(AEC, self).__init__()
        self.EnC = torch.nn.Sequential(
            nn.Linear(IC_dim, h_dim2),
            nn.BatchNorm1d(h_dim2),
            nn.ReLU(),
            nn.Dropout(rate2))
    def forward(self, x):
        output = self.EnC(x)
        return output       

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.FC = torch.nn.Sequential(
            nn.Linear(Z_in, 1),
            nn.Dropout(rate3),
            nn.Sigmoid())
    def forward(self, x):
        return self.FC(x)

torch.cuda.manual_seed_all(42)

AutoencoderE = torch.load(models_dir + "Exprs_Erlotinib_GDSC_PDX_EGFRi_Expression_CNA.pt")
AutoencoderC = torch.load(models_dir + "CNA_Erlotinib_GDSC_PDX_EGFRi_Expression_CNA.pt")

Clas = torch.load(models_dir + "Class_Erlotinib_GDSC_PDX_EGFRi_Expression_CNA.pt")

AutoencoderE.eval()
AutoencoderC.eval()
Clas.eval()

ZEX = AutoencoderE(torch.FloatTensor(X_trainE))
ZCX = AutoencoderC(torch.FloatTensor(X_trainC))
ZTX = torch.cat((ZEX, ZCX), 1)
ZTX = F.normalize(ZTX, p=2, dim=0)
PredX = Clas(ZTX)
AUCT = roc_auc_score(Y, PredX.detach().numpy())
#print(f"GDSC - Training AUC (PDX_EGFRi): {round(AUCT,2)}")
AUCTaucpr = average_precision_score(Y, PredX.detach().numpy())
#print(f"GDSC - Training AUCPR (PDX_EGFRi): {round(AUCTaucpr,2)}\n")

ZETerlotinib = AutoencoderE(TX_testEerlotinib)
ZCTerlotinib = AutoencoderC(TX_testCerlotinib)
ZTTerlotinib = torch.cat((ZETerlotinib, ZCTerlotinib), 1)
ZTTerlotinib = F.normalize(ZTTerlotinib, p=2, dim=0)
PredTerlotinib = Clas(ZTTerlotinib)
AUCTerlotinib = roc_auc_score(Ytserlotinib.astype(int), PredTerlotinib.detach().numpy())
#print(f"PDX - Test AUC (Erlotinib): {round(AUCTerlotinib,2)}")
AUCTerlotinib_aucpr = average_precision_score(Ytserlotinib.astype(int), PredTerlotinib.detach().numpy())
#print(f"PDX - Test AUCPR (Erlotinib): {round(AUCTerlotinib_aucpr,2)}\n")

ZETerlotinib_nsclc = AutoencoderE(TX_testEerlotinib_nsclc)
ZCTerlotinib_nsclc = AutoencoderC(TX_testCerlotinib_nsclc)
ZTTerlotinib_nsclc = torch.cat((ZETerlotinib_nsclc, ZCTerlotinib_nsclc), 1)
ZTTerlotinib_nsclc = F.normalize(ZTTerlotinib_nsclc, p=2, dim=0)
PredTerlotinib_nsclc = Clas(ZTTerlotinib_nsclc)
AUCTerlotinib_nsclc = roc_auc_score(Ytserlotinib_nsclc.astype(int), PredTerlotinib_nsclc.detach().numpy())
#print(f"PDX - Test AUC (Erlotinib, NSCLC): {round(AUCTerlotinib_nsclc,2)}")
AUCTerlotinib_nsclc_aucpr = average_precision_score(Ytserlotinib_nsclc.astype(int), PredTerlotinib_nsclc.detach().numpy())
#print(f"PDX - Test AUCPR (Erlotinib, NSCLC): {round(AUCTerlotinib_nsclc_aucpr,2)}\n")

AUCT_finetuned = dict()
AUCTaucpr_finetuned = dict()
AUCTerlotinib_finetuned = dict()
AUCTerlotinib_nsclc_finetuned = dict()
AUCTerlotinib_aucpr_finetuned = dict()
AUCTerlotinib_nsclc_aucpr_finetuned = dict()
for iter in range(max_iter):
   iter += 1  

   AutoencoderE_finetuned = torch.load(models_dir + f"Finetuned_Models/Exprs_Erlotinib_GDSC_PDX_EGFRi_Expression_CNA_Finetuned_{iter}.pt")
   AutoencoderC_finetuned = torch.load(models_dir + f"Finetuned_Models/CNA_Erlotinib_GDSC_PDX_EGFRi_Expression_CNA_Finetuned_{iter}.pt")
   Clas_finetuned = torch.load(models_dir + f"Finetuned_Models/Class_Erlotinib_GDSC_PDX_EGFRi_Expression_CNA_Finetuned_{iter}.pt")
   
   AutoencoderE_finetuned.eval()
   AutoencoderC_finetuned.eval()
   Clas_finetuned.eval()
      
   ZEX = AutoencoderE_finetuned(torch.FloatTensor(X_trainE))
   ZCX = AutoencoderC_finetuned(torch.FloatTensor(X_trainC))
   ZTX = torch.cat((ZEX, ZCX), 1)
   ZTX = F.normalize(ZTX, p=2, dim=0)
   PredX = Clas_finetuned(ZTX)
   AUCT_finetuned[iter] = roc_auc_score(Y, PredX.detach().numpy())
   AUCTaucpr_finetuned[iter] = average_precision_score(Y, PredX.detach().numpy())
   
   ZETerlotinib = AutoencoderE_finetuned(TX_testEerlotinib)
   ZCTerlotinib = AutoencoderC_finetuned(TX_testCerlotinib)
   ZTTerlotinib = torch.cat((ZETerlotinib, ZCTerlotinib), 1)
   ZTTerlotinib = F.normalize(ZTTerlotinib, p=2, dim=0)
   PredTerlotinib = Clas_finetuned(ZTTerlotinib)
   AUCTerlotinib_finetuned[iter] = roc_auc_score(Ytserlotinib.astype(int), PredTerlotinib.detach().numpy())
   AUCTerlotinib_aucpr_finetuned[iter] = average_precision_score(Ytserlotinib.astype(int), PredTerlotinib.detach().numpy())

   ZETerlotinib_nsclc = AutoencoderE_finetuned(TX_testEerlotinib_nsclc)
   ZCTerlotinib_nsclc = AutoencoderC_finetuned(TX_testCerlotinib_nsclc)
   ZTTerlotinib_nsclc = torch.cat((ZETerlotinib_nsclc, ZCTerlotinib_nsclc), 1)
   ZTTerlotinib_nsclc = F.normalize(ZTTerlotinib_nsclc, p=2, dim=0)
   PredTerlotinib_nsclc = Clas_finetuned(ZTTerlotinib_nsclc)
   AUCTerlotinib_nsclc_finetuned[iter] = roc_auc_score(Ytserlotinib_nsclc.astype(int), PredTerlotinib_nsclc.detach().numpy())
   AUCTerlotinib_nsclc_aucpr_finetuned[iter] = average_precision_score(Ytserlotinib_nsclc.astype(int), PredTerlotinib_nsclc.detach().numpy())

max_key = max(AUCTerlotinib_aucpr_finetuned, key=AUCTerlotinib_aucpr_finetuned.get)
max_key_nsclc = max(AUCTerlotinib_nsclc_aucpr_finetuned, key=AUCTerlotinib_nsclc_aucpr_finetuned.get)

print(f"GDSC - Training AUC (EGFRi): {round(AUCT,2)}")
print(f"GDSC - Training AUCPR (EGFRi): {round(AUCTaucpr,2)}")
print(f"PDX - Test AUC (Erlotinib): {round(AUCTerlotinib,2)}")
print(f"PDX - Test AUCPR (Erlotinib): {round(AUCTerlotinib_aucpr,2)}")
print(f"PDX - Test AUC (Erlotinib, NSCLC): {round(AUCTerlotinib_nsclc,2)}")
print(f"PDX - Test AUCPR (Erlotinib, NSCLC): {round(AUCTerlotinib_nsclc_aucpr,2)}\n")

print(f"GDSC - Training AUC (EGFRi, Finetuned): {round(AUCT_finetuned[max_key],2)}")
print(f"GDSC - Training AUCPR (EGFRi, Finetuned): {round(AUCTaucpr_finetuned[max_key],2)}")
print(f"PDX - Test AUC (Erlotinib, Finetuned): {round(AUCTerlotinib_finetuned[max_key],2)}")
print(f"PDX - Test AUCPR (Erlotinib, Finetuned): {round(AUCTerlotinib_aucpr_finetuned[max_key],2)}\n")


print(f"GDSC - Training AUC (EGFRi, Finetuned): {round(AUCT_finetuned[max_key_nsclc],2)}")
print(f"GDSC - Training AUCPR (EGFRi, Finetuned): {round(AUCTaucpr_finetuned[max_key_nsclc],2)}")
print(f"PDX - Test AUC (Erlotinib, NSCLC, Finetuned): {round(AUCTerlotinib_nsclc_finetuned[max_key_nsclc],2)}")
print(f"PDX - Test AUCPR (Erlotinib, NSCLC, Finetuned): {round(AUCTerlotinib_nsclc_aucpr_finetuned[max_key_nsclc],2)}\n\n")

print("Models for maximum finetuning (Erlotinib)")
print(f"Exprs_Erlotinib_GDSC_PDX_EGFRi_Expression_CNA_Finetuned_{max_key}.pt")
print(f"CNA_Erlotinib_GDSC_PDX_EGFRi_Expression_CNA_Finetuned_{max_key}.pt")
print(f"Class_Erlotinib_GDSC_PDX_EGFRi_Expression_CNA_Finetuned_{max_key}.pt")


print("\nModels for maximum finetuning (Erlotinib, NSCLC)")
print(f"Exprs_Erlotinib_GDSC_PDX_EGFRi_Expression_CNA_Finetuned_{max_key_nsclc}.pt")
print(f"CNA_Erlotinib_GDSC_PDX_EGFRi_Expression_CNA_Finetuned_{max_key_nsclc}.pt")
print(f"Class_Erlotinib_GDSC_PDX_EGFRi_Expression_CNA_Finetuned_{max_key_nsclc}.pt")

GDSC - Training AUC (EGFRi): 0.78
GDSC - Training AUCPR (EGFRi): 0.34
PDX - Test AUC (Erlotinib): 0.81
PDX - Test AUCPR (Erlotinib): 0.48
PDX - Test AUC (Erlotinib, NSCLC): 0.81
PDX - Test AUCPR (Erlotinib, NSCLC): 0.48

GDSC - Training AUC (EGFRi, Finetuned): 0.81
GDSC - Training AUCPR (EGFRi, Finetuned): 0.43
PDX - Test AUC (Erlotinib, Finetuned): 0.85
PDX - Test AUCPR (Erlotinib, Finetuned): 0.61

GDSC - Training AUC (EGFRi, Finetuned): 0.81
GDSC - Training AUCPR (EGFRi, Finetuned): 0.43
PDX - Test AUC (Erlotinib, NSCLC, Finetuned): 0.85
PDX - Test AUCPR (Erlotinib, NSCLC, Finetuned): 0.61


Models for maximum finetuning (Erlotinib)
Exprs_Erlotinib_GDSC_PDX_EGFRi_Expression_CNA_Finetuned_11.pt
CNA_Erlotinib_GDSC_PDX_EGFRi_Expression_CNA_Finetuned_11.pt
Class_Erlotinib_GDSC_PDX_EGFRi_Expression_CNA_Finetuned_11.pt

Models for maximum finetuning (Erlotinib, NSCLC)
Exprs_Erlotinib_GDSC_PDX_EGFRi_Expression_CNA_Finetuned_11.pt
CNA_Erlotinib_GDSC_PDX_EGFRi_Expression_CNA_Finetuned_11.pt


In [42]:
# show expression layer parameters

AutoencoderE

AEE(
  (EnE): Sequential(
    (0): Linear(in_features=1019, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
  )
)

In [43]:
# show CNA layer parameters

AutoencoderC

AEC(
  (EnC): Sequential(
    (0): Linear(in_features=1019, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.8, inplace=False)
  )
)

In [44]:
# show classification layer parameters

Clas

Classifier(
  (FC): Sequential(
    (0): Linear(in_features=768, out_features=1, bias=True)
    (1): Dropout(p=0.8, inplace=False)
    (2): Sigmoid()
  )
)

In [45]:
# create a dataframe with the prediction results 

ec = {"Data": ["Expr + CNA", "Expr + CNA (Fine-tuned)"],
       "AUC (GDSC)": [round(AUCT, 2), round(AUCT_finetuned[max_key], 2)],
       "AUCPR (GDSC)": [round(AUCTaucpr,2), round(AUCTaucpr_finetuned[max_key],2)],
       "AUC (PDX)": [round(AUCTerlotinib,2), round(AUCTerlotinib_finetuned[max_key],2)],
       "AUCPR (PDX)":  [round(AUCTerlotinib_aucpr,2), round(AUCTerlotinib_aucpr_finetuned[max_key],2)],
       "AUC (PDX-NSCLC)":  [round(AUCTerlotinib_nsclc,2), round(AUCTerlotinib_nsclc_finetuned[max_key_nsclc],2)],
       "AUCPR (PDX-NSCLC)": [round(AUCTerlotinib_nsclc_aucpr,2), round(AUCTerlotinib_nsclc_aucpr_finetuned[max_key_nsclc],2)],
       "Sample Size (GDSC)": [GDSCEv2.shape[0], GDSCEv2.shape[0]],
       "Feature Size (GDSC)": [GDSCEv2.shape[1], GDSCEv2.shape[1]] 
}
ec_dataframe = pd.DataFrame.from_dict(ec, orient='index').transpose()  
ec_dataframe.to_csv(save_results_to + "GDSC_PDX_Expression_CNA_EGFRi_Erlotinib.tsv",
                    sep = "\t",
                    index = False)

In [46]:
# change directory

os.chdir(save_results_to)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/PDX_EGFRi/Expression_CNA/Predictions


In [47]:
# save predictions

file = open('GDSC_PDX_DEGs - PDX_EGFRi_Erlotinib Predictions (Expression and CNA).txt', 'w')
file.write(f"GDSC Training (EC) AUC (PDX_EGFRi): {round(AUCT,2)}\n")
file.write(f"GDSC Training (EC) AUCPR (PDX_EGFRi): {round(AUCTaucpr,2)}\n\n")

file.write(f"PDX Test (EC) AUC (Erlotinib): {round(AUCTerlotinib,2)}\n")
file.write(f"PDX Test (EC) AUCPR (Erlotinib): {round(AUCTerlotinib_aucpr,2)}\n\n")

file.close()